In [7]:
import pandas as pd
import os
from google.cloud import bigquery
import json
from google.oauth2 import service_account

In [8]:
# Obtain the key from the environment variable
service_account_key =os.getenv('GCP_ACCESS_KEY')
if not service_account_key:
    raise ValueError("The GCP_SERVICE_ACCOUNT_KEY environment variable is not set")

key_data = json.loads(service_account_key)

credentials = service_account.Credentials.from_service_account_info(
    key_data,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)


In [4]:
service_acc_key=os.getenv('GCP_ACCESS_KEY')

In [9]:
client = bigquery.Client(credentials=credentials)